In [1]:
from pprint import pprint
from collections import defaultdict
import json
import re
import sys
from typing import Union, List, Optional, Set, Tuple, Dict, Optional, Callable
from pprint import pprint
from IPython.display import display, HTML

import numpy as np
from lab.utils import shorten
import pandas as pd
from pathlib import Path
from machine_learning.analysis.dataframe import (
    pivot_rotate,

    slice_rows,
    slice_cols,
    sort_rows,
    sort_cols,
    aggregate,
    percentize,
    round,

    rename_index,
    rename_cols,
    rename_cells,

    isnan,
    to_latex,
    color_by_rank,
)
from machine_learning.analysis.series import (
    maybe_numeric_series,
)
from machine_learning.analysis.utils import (
    maybe_round,
)
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', 1000)

/home/acb11878tj/.pyenv/versions/3.8.5/envs/jupyter-analysis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def find_result_paths(top_dir: Union[str, Path], regexps: Optional[List[str]] = None) -> List[str]:
    regexps = regexps or []
    top_dir = Path(top_dir)
    return [
        str(path) for path in top_dir.glob('**/results.tsv')
        if all(re.match(regexp, str(path)) for regexp in regexps)
    ]

In [3]:
def name_method(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
            
    df[COL_METHOD] = df.apply(
        lambda row: '__'.join(str(row[col]) for col in METHOD_DEFINE_COLS),
        axis=1,
    )

    df = sort_cols(df, [COL_METHOD, '.*'])

    return df

def name_task(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    df[COL_TASK] = df.apply(
        lambda row: '__'.join(str(row[col]) for col in TASK_DEFINE_COLS),
        axis=1,
    )

    df = sort_cols(df, [COL_TASK, '.*'])
    
    return df

def prettify_df(df: pd.DataFrame) -> pd.DataFrame:
    df = percentize(df)
    df = round(df)
    return df

In [4]:
COL_DATASET = 'dataset_uname'
COL_LEARNING = 'learning'
COL_MODEL_NAME_OR_PATH = 'model_name_or_path'
COL_LRATE = 'learning_rate'

TASK_DEFINE_COLS = [COL_DATASET, COL_LEARNING]
COL_TASK = 'task'

METHOD_DEFINE_COLS = [COL_MODEL_NAME_OR_PATH]
COL_METHOD = 'method'

COL_LRATE = 'learning_rate'
LRATE = 0.0001

METRIC_RENAMES = {
    'eval/extr_stps.D-all.proof_accuracy.zero_one': 'prf_acc.extr',
    # 'eval/strct.D-all.proof_accuracy.zero_one': 'prf_acc.strct',
    # 'eval/strct.D-all.answer_accuracy': 'ans.acc',
}
METRIC_NAMES = list(METRIC_RENAMES.values())

TASK_RENAMES = {
    '^20230904.jpn.D1.wo_brnch.wo_dstrct__LLM_FS.shot-10$': 'D1--.10',
    '^20230904.jpn.D1.wo_brnch.wo_dstrct__LLM_FS.shot-100$': 'D1--.100',
    '^20230904.jpn.D1.wo_brnch.wo_dstrct__LLM_FS.shot-1000$': 'D1--.1000',
    
    '^20230904.jpn.D1.wo_brnch__LLM_FS.shot-10$': 'D1-.10',
    '^20230904.jpn.D1.wo_brnch__LLM_FS.shot-100$': 'D1-.100',
    '^20230904.jpn.D1.wo_brnch__LLM_FS.shot-1000$': 'D1-.1000',
    
    '^20230904.jpn.D1__LLM_FS.shot-10$': 'D1.10',
    '^20230904.jpn.D1__LLM_FS.shot-100$': 'D1.100',
    '^20230904.jpn.D1__LLM_FS.shot-1000$': 'D1.1000',
    
    '^20230904.jpn.D3__LLM_FS.shot-10$': 'D3.10',
    '^20230904.jpn.D3__LLM_FS.shot-100$': 'D3.100',
    '^20230904.jpn.D3__LLM_FS.shot-1000$': 'D3.1000',
}
TASK_NAMES = list(TASK_RENAMES.values())
MAJOR_TASK = TASK_NAMES[1]

In [5]:
# _TOP_DIR = '../outputs/02.aggregate_tf_results.py/20230826.jpn/'
# _TOP_DIR = '../outputs/02.aggregate_tf_results.py/20230901.overfit/'
# _TOP_DIR = '../outputs/02.aggregate_tf_results.py/20230904.LLM_FS/'
_TOP_DIR = '../outputs/02.aggregate_tf_results.py/20230905.LLM_FS/'

In [6]:
result_paths = find_result_paths(_TOP_DIR)
if len(result_paths) == 0:
    raise Exception(f'Results not found under {_TOP_DIR}')
elif len(result_paths) == 1:
    results_path = result_paths[0]
else:
    print('Choose the result fomr the following paths:')
    pprint(result_paths)
    results_path = input('path = ')
    


In [7]:
master_df = pd.read_csv(results_path, sep='\t')
master_df.head()

,dataset_uname,learning,model_name_or_path,seed,learning_rate,base_config_name,generation_max_proof_steps,generation_num_beams,generation_top_k,gradient_accumulation_steps,lm_type,lora,max_grad_norm,max_predict_samples,max_proof_steps,max_source_length,max_steps,max_target_length,max_train_samples,per_device_eval_batch_size,per_device_train_batch_size,proof_sampling,sample_negative_proof,source_prefix,tokenizer_padding,warmup_steps,eval/extr_stps.D-0.proof_accuracy.zero_one,eval/extr_stps.D-1.proof_accuracy.zero_one,eval/extr_stps.D-2.proof_accuracy.zero_one,eval/extr_stps.D-3.proof_accuracy.zero_one,eval/extr_stps.D-4.proof_accuracy.zero_one,eval/extr_stps.D-5.proof_accuracy.zero_one,eval/extr_stps.D-6.proof_accuracy.zero_one,eval/extr_stps.D-7.proof_accuracy.zero_one,eval/extr_stps.D-8.proof_accuracy.zero_one,eval/extr_stps.D-None.proof_accuracy.zero_one,eval/extr_stps.D-all.proof_accuracy.zero_one,eval/strct.D-0.proof_accuracy.zero_one,eval/strct.D-1.proof_accuracy.zero_one,eval/strct.D-2.proof_accuracy.zero_one,eval/strct.D-3.proof_accuracy.zero_one,eval/strct.D-4.proof_accuracy.zero_one,eval/strct.D-5.proof_accuracy.zero_one,eval/strct.D-6.proof_accuracy.zero_one,eval/strct.D-7.proof_accuracy.zero_one,eval/strct.D-8.proof_accuracy.zero_one,eval/strct.D-None.proof_accuracy.zero_one,eval/strct.D-all.proof_accuracy.zero_one,eval/extr_stps.D-0.answer_accuracy,eval/extr_stps.D-1.answer_accuracy,eval/extr_stps.D-2.answer_accuracy,eval/extr_stps.D-3.answer_accuracy,eval/extr_stps.D-4.answer_accuracy,eval/extr_stps.D-5.answer_accuracy,eval/extr_stps.D-6.answer_accuracy,eval/extr_stps.D-7.answer_accuracy,eval/extr_stps.D-8.answer_accuracy,eval/extr_stps.D-None.answer_accuracy,eval/extr_stps.D-all.answer_accuracy,eval/strct.D-0.answer_accuracy,eval/strct.D-1.answer_accuracy,eval/strct.D-2.answer_accuracy,eval/strct.D-3.answer_accuracy,eval/strct.D-4.answer_accuracy,eval/strct.D-5.answer_accuracy,eval/strct.D-6.answer_accuracy,eval/strct.D-7.answer_accuracy,eval/strct.D-8.answer_accuracy,eval/strct.D-None.answer_accuracy,eval/strct.D-all.answer_accuracy
0,20230904.jpn.D1.wo_brnch,LLM_FS.shot-100,matsuo-lab/weblab-10b,1,0.00010,FLNLcorpus.20220827.base,20,1,10,4,causal,True,0.5,0,30,2000,170,2000,100,1,2,all_at_once,True,Solve FLD task:,longest,17,0.0,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.033333,0.0,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.033333,0.666667,0.764706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.500000,0.666667,0.764706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.500000
1,20230904.jpn.D1.wo_brnch,LLM_FS.shot-100,stabilityai/japanese-stablelm-base-alpha-7b,0,0.00010,FLNLcorpus.20220827.base,20,1,10,4,causal,True,0.5,0,30,2000,170,2000,100,1,2,all_at_once,True,Solve FLD task:,longest,17,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,0.033333,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,0.666667,0.529412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,0.400000,0.666667,0.529412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,0.400000
2,20230904.jpn.D1.wo_brnch,LLM_FS.shot-100,matsuo-lab/weblab-10b,0,0.00010,FLNLcorpus.20220827.base,20,1,10,4,causal,True,0.5,0,30,2000,320,2000,100,1,2,all_at_once,True,Solve FLD task:,longest,32,0.0,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,0.066667,0.0,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.033333,0.666667,0.647059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,0.466667,0.666667,0.647059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,0.466667
3,20230904.jpn.D1.wo_brnch,LLM_FS.shot-100,matsuo-lab/weblab-10b,0,0.00001,FLNLcorpus.20220827.base,20,1,10,4,causal,True,0.5,0,30,2000,320,2000,100,1,2,all_at_once,True,Solve FLD task:,longest,32,0.0,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,0.066667,0.0,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.033333,1.000000,0.647059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,0.500000,1.000000,0.647059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,0.500000
4,20230904.jpn.D1.wo_brnch,LLM_FS.shot-10,matsuo-lab/weblab-10b,0,0.00010,FLNLcorpus.20220827.base,20,1,10,4,causal,True,0.5,0,30,2000,70,2000,10,1,2,all_at_once,True,Solve FLD task:,longest,7,0.0,0.000

In [8]:
df = name_method(master_df)

df = name_task(df)
df = rename_cells(df, [COL_TASK], TASK_RENAMES)

df = rename_cols(df, METRIC_RENAMES)
                 
df = slice_rows(df, lambda row: row[COL_LRATE] == LRATE)
df = aggregate(
    df,
    [COL_METHOD, COL_TASK],
    {metric_name: lambda vals: np.mean(vals) for metric_name in METRIC_NAMES},
)

df

,method,task,dataset_uname,learning,model_name_or_path,seed,learning_rate,base_config_name,generation_max_proof_steps,generation_num_beams,generation_top_k,gradient_accumulation_steps,lm_type,lora,max_grad_norm,max_predict_samples,max_proof_steps,max_source_length,max_steps,max_target_length,max_train_samples,per_device_eval_batch_size,per_device_train_batch_size,proof_sampling,sample_negative_proof,source_prefix,tokenizer_padding,warmup_steps,eval/extr_stps.D-0.proof_accuracy.zero_one,eval/extr_stps.D-1.proof_accuracy.zero_one,eval/extr_stps.D-2.proof_accuracy.zero_one,eval/extr_stps.D-3.proof_accuracy.zero_one,eval/extr_stps.D-4.proof_accuracy.zero_one,eval/extr_stps.D-5.proof_accuracy.zero_one,eval/extr_stps.D-6.proof_accuracy.zero_one,eval/extr_stps.D-7.proof_accuracy.zero_one,eval/extr_stps.D-8.proof_accuracy.zero_one,eval/extr_stps.D-None.proof_accuracy.zero_one,prf_acc.extr,eval/strct.D-0.proof_accuracy.zero_one,eval/strct.D-1.proof_accuracy.zero_one,eval/strct.D-2.proof_accuracy.zero_one,eval/strct.D-3.proof_accuracy.zero_one,eval/strct.D-4.proof_accuracy.zero_one,eval/strct.D-5.proof_accuracy.zero_one,eval/strct.D-6.proof_accuracy.zero_one,eval/strct.D-7.proof_accuracy.zero_one,eval/strct.D-8.proof_accuracy.zero_one,eval/strct.D-None.proof_accuracy.zero_one,eval/strct.D-all.proof_accuracy.zero_one,eval/extr_stps.D-0.answer_accuracy,eval/extr_stps.D-1.answer_accuracy,eval/extr_stps.D-2.answer_accuracy,eval/extr_stps.D-3.answer_accuracy,eval/extr_stps.D-4.answer_accuracy,eval/extr_stps.D-5.answer_accuracy,eval/extr_stps.D-6.answer_accuracy,eval/extr_stps.D-7.answer_accuracy,eval/extr_stps.D-8.answer_accuracy,eval/extr_stps.D-None.answer_accuracy,eval/extr_stps.D-all.answer_accuracy,eval/strct.D-0.answer_accuracy,eval/strct.D-1.answer_accuracy,eval/strct.D-2.answer_accuracy,eval/strct.D-3.answer_accuracy,eval/strct.D-4.answer_accuracy,eval/strct.D-5.answer_accuracy,eval/strct.D-6.answer_accuracy,eval/strct.D-7.answer_accuracy,eval/strct.D-8.answer_accuracy,eval/strct.D-None.answer_accuracy,eval/strct.D-all.answer_accuracy
"(matsuo-lab/weblab-10b, D1-.100)",matsuo-lab/weblab-10b,D1-.100,20230904.jpn.D1.wo_brnch,LLM_FS.shot-100,matsuo-lab/weblab-10b,1,0.0001,FLNLcorpus.20220827.base,20,1,10,4,causal,True,0.5,0,30,2000,170,2000,100,1,2,all_at_once,True,Solve FLD task:,longest,17,0.000000,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.050000,0.000000,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.033333,0.666667,0.764706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.500000,0.666667,0.764706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.500000
"(stabilityai/japanese-stablelm-base-alpha-7b, D1-.100)",stabilityai/japanese-stablelm-base-alpha-7b,D1-.100,20230904.jpn.D1.wo_brnch,LLM_FS.shot-100,stabilityai/japanese-stablelm-base-alpha-7b,0,0.0001,FLNLcorpus.20220827.base,20,1,10,4,causal,True,0.5,0,30,2000,170,2000,100,1,2,all_at_once,True,Solve FLD task:,longest,17,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100000,0.033333,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.666667,0.529412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100000,0.400000,0.666667,0.529412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100000,0.400000
"(matsuo-lab/weblab-10b, D1-.10)",matsuo-lab/weblab-10b,D1-.10,20230904.jpn.D1.wo_brnch,LLM_FS.shot-10,matsuo-lab/weblab-10b,0,0.0001,FLNLcorpus.20220827.base,20,1,10,4,causal,True,0.5,0,30,2000,70,2000,10,1,2,all_at_once,True,Solve FLD task:,longest,7,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.117647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.066667,0.000000,0.117647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.066667
"(stabilityai/japanese-stablelm-base-alpha-7b, D1-.10)",stabilityai/japanese-stablelm-base-alpha-7b,D1-.10,20230904.jpn.D1.wo_brnch,LLM_FS.shot-10,stabilityai/japanese-stablelm-base-alpha-7b,0,0.0001,FLNLcorpus.20220827.base,20,1,10,4,causal,True,0.5,0,30,2000,70,2000,10,1,2,all_at_once,True,Solve FLD task:,longest,7,0.000000,0.0

In [9]:
for task in df[COL_TASK].unique():
    print("'" + task + "',")

'D1-.100',
'D1-.10',
'D1-.1000',
'D1--.100',
'D1--.10',
'D1--.1000',
'D1.100',
'D1.10',
'D1.1000',
'D3.100',
'D3.10',
'D3.1000',


In [10]:
task_dfs: Dict[str, pd.DataFrame] = {}
for task_name in TASK_NAMES:
    task_dfs[task_name] = slice_rows(
        df,
        lambda row: row[COL_TASK] == task_name
    )

task_dfs[MAJOR_TASK]

,method,task,dataset_uname,learning,model_name_or_path,seed,learning_rate,base_config_name,generation_max_proof_steps,generation_num_beams,generation_top_k,gradient_accumulation_steps,lm_type,lora,max_grad_norm,max_predict_samples,max_proof_steps,max_source_length,max_steps,max_target_length,max_train_samples,per_device_eval_batch_size,per_device_train_batch_size,proof_sampling,sample_negative_proof,source_prefix,tokenizer_padding,warmup_steps,eval/extr_stps.D-0.proof_accuracy.zero_one,eval/extr_stps.D-1.proof_accuracy.zero_one,eval/extr_stps.D-2.proof_accuracy.zero_one,eval/extr_stps.D-3.proof_accuracy.zero_one,eval/extr_stps.D-4.proof_accuracy.zero_one,eval/extr_stps.D-5.proof_accuracy.zero_one,eval/extr_stps.D-6.proof_accuracy.zero_one,eval/extr_stps.D-7.proof_accuracy.zero_one,eval/extr_stps.D-8.proof_accuracy.zero_one,eval/extr_stps.D-None.proof_accuracy.zero_one,prf_acc.extr,eval/strct.D-0.proof_accuracy.zero_one,eval/strct.D-1.proof_accuracy.zero_one,eval/strct.D-2.proof_accuracy.zero_one,eval/strct.D-3.proof_accuracy.zero_one,eval/strct.D-4.proof_accuracy.zero_one,eval/strct.D-5.proof_accuracy.zero_one,eval/strct.D-6.proof_accuracy.zero_one,eval/strct.D-7.proof_accuracy.zero_one,eval/strct.D-8.proof_accuracy.zero_one,eval/strct.D-None.proof_accuracy.zero_one,eval/strct.D-all.proof_accuracy.zero_one,eval/extr_stps.D-0.answer_accuracy,eval/extr_stps.D-1.answer_accuracy,eval/extr_stps.D-2.answer_accuracy,eval/extr_stps.D-3.answer_accuracy,eval/extr_stps.D-4.answer_accuracy,eval/extr_stps.D-5.answer_accuracy,eval/extr_stps.D-6.answer_accuracy,eval/extr_stps.D-7.answer_accuracy,eval/extr_stps.D-8.answer_accuracy,eval/extr_stps.D-None.answer_accuracy,eval/extr_stps.D-all.answer_accuracy,eval/strct.D-0.answer_accuracy,eval/strct.D-1.answer_accuracy,eval/strct.D-2.answer_accuracy,eval/strct.D-3.answer_accuracy,eval/strct.D-4.answer_accuracy,eval/strct.D-5.answer_accuracy,eval/strct.D-6.answer_accuracy,eval/strct.D-7.answer_accuracy,eval/strct.D-8.answer_accuracy,eval/strct.D-None.answer_accuracy,eval/strct.D-all.answer_accuracy
"(matsuo-lab/weblab-10b, D1--.100)",matsuo-lab/weblab-10b,D1--.100,20230904.jpn.D1.wo_brnch.wo_dstrct,LLM_FS.shot-100,matsuo-lab/weblab-10b,1,0.0001,FLNLcorpus.20220827.base,20,1,10,4,causal,True,0.5,0,30,2000,170,2000,100,1,2,all_at_once,True,Solve FLD task:,longest,17,0.0,0.619048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.433333,0.0,0.619048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.433333,0.5,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.500000,0.5,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.500000
"(stabilityai/japanese-stablelm-base-alpha-7b, D1--.100)",stabilityai/japanese-stablelm-base-alpha-7b,D1--.100,20230904.jpn.D1.wo_brnch.wo_dstrct,LLM_FS.shot-100,stabilityai/japanese-stablelm-base-alpha-7b,0,0.0001,FLNLcorpus.20220827.base,20,1,10,4,causal,True,0.5,0,30,2000,170,2000,100,1,2,all_at_once,True,Solve FLD task:,longest,17,0.5,0.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.433333,0.5,0.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.433333,1.0,0.714286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.566667,1.0,0.714286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.566667


In [11]:

metric_dfs: Dict[str, pd.DataFrame] = {}
for task_name, task_df in task_dfs.items():
    metric_dfs[task_name] = slice_cols(task_df, [COL_TASK, COL_METHOD] + METRIC_NAMES)

for task_name in TASK_NAMES:
    print('\n\n')
    print(f'========================= {task_name} ========================')
    display(metric_dfs[task_name])




========================= D1--.10 ========================


,task,method,prf_acc.extr
"(matsuo-lab/weblab-10b, D1--.10)",D1--.10,matsuo-lab/weblab-10b,0.183333
"(stabilityai/japanese-stablelm-base-alpha-7b, D1--.10)",D1--.10,stabilityai/japanese-stablelm-base-alpha-7b,0.200000





========================= D1--.100 ========================


,task,method,prf_acc.extr
"(matsuo-lab/weblab-10b, D1--.100)",D1--.100,matsuo-lab/weblab-10b,0.433333
"(stabilityai/japanese-stablelm-base-alpha-7b, D1--.100)",D1--.100,stabilityai/japanese-stablelm-base-alpha-7b,0.433333





========================= D1--.1000 ========================


...

,task,method,prf_acc.extr
"(matsuo-lab/weblab-10b, D3.10)",D3.10,matsuo-lab/weblab-10b,0.0





========================= D3.100 ========================


,task,method,prf_acc.extr
"(matsuo-lab/weblab-10b, D3.100)",D3.100,matsuo-lab/weblab-10b,0.0





========================= D3.1000 ========================


,task,method,prf_acc.extr
"(matsuo-lab/weblab-10b, D3.1000)",D3.1000,matsuo-lab/weblab-10b,0.066667


In [12]:

pretty_dfs: Dict[str, pd.DataFrame] = {}

for task_name, metric_df in metric_dfs.items():
    pretty_df = prettify_df(metric_df)
    pretty_df = rename_cols(pretty_df, METRIC_RENAMES)
    
    pretty_df.index = pretty_df[COL_METHOD]
    pretty_df = pretty_df.drop(columns=[COL_METHOD])
    
    pretty_df = color_by_rank(pretty_df, 'col')
    
    pretty_dfs[task_name] = pretty_df


for task_name in TASK_NAMES:
    print('\n\n')
    print(f'========================= {task_name} ========================')
    display(pretty_dfs[task_name])




========================= D1--.10 ========================


,task,prf_acc.extr
method,,
matsuo-lab/weblab-10b,D1--.10,\cellcolor{blue!10} 18.3
stabilityai/japanese-stablelm-base-alpha-7b,D1--.10,\cellcolor{blue!60} 20.0





========================= D1--.100 ========================


,task,prf_acc.extr
method,,
matsuo-lab/weblab-10b,D1--.100,\cellcolor{blue!35} 43.3
stabilityai/japanese-stablelm-base-alpha-7b,D1--.100,\cellcolor{blue!35} 43.3





========================= D1--.1000 ========================


...

,task,prf_acc.extr
method,,
matsuo-lab/weblab-10b,D3.10,\cellcolor{blue!35} 0.0





========================= D3.100 ========================


,task,prf_acc.extr
method,,
matsuo-lab/weblab-10b,D3.100,\cellcolor{blue!35} 0.0





========================= D3.1000 ========================


,task,prf_acc.extr
method,,
matsuo-lab/weblab-10b,D3.1000,\cellcolor{blue!35} 6.7


In [13]:
def horizontal_concat(dfs: List[pd.DataFrame]) -> pd.DataFrame:
    dfs = [df.copy() for df in dfs]
    # align index for horizontal concat
    # for df in dfs:
    #     df.index = range(len(df))
    return pd.concat(dfs, axis=1)

colored_concat_df = horizontal_concat(
    [pretty_df for task_name, pretty_df in pretty_dfs.items()]
)
colored_concat_df = colored_concat_df.drop(columns=[COL_TASK], axis=1)

print('    '.join([task_name for task_name in pretty_dfs.keys()]))
colored_concat_df

D1--.10    D1--.100    D1--.1000    D1-.10    D1-.100    D1-.1000    D1.10    D1.100    D1.1000    D3.10    D3.100    D3.1000


,prf_acc.extr,prf_acc.extr,prf_acc.extr,prf_acc.extr,prf_acc.extr,prf_acc.extr,prf_acc.extr,prf_acc.extr,prf_acc.extr,prf_acc.extr,prf_acc.extr,prf_acc.extr
method,,,,,,,,,,,,
matsuo-lab/weblab-10b,\cellcolor{blue!10} 18.3,\cellcolor{blue!35} 43.3,\cellcolor{blue!60} 76.7,\cellcolor{blue!10} 0.0,\cellcolor{blue!60} 5.0,\cellcolor{blue!60} 23.3,\cellcolor{blue!10} 0.0,\cellcolor{blue!35} 10.0,\cellcolor{blue!60} 30.0,\cellcolor{blue!35} 0.0,\cellcolor{blue!35} 0.0,\cellcolor{blue!35} 6.7
stabilityai/japanese-stablelm-base-alpha-7b,\cellcolor{blue!60} 20.0,\cellcolor{blue!35} 43.3,\cellcolor{blue!10} 60.0,\cellcolor{blue!60} 3.3,\cellcolor{blue!10} 3.3,\cellcolor{blue!10} 0.0,\cellcolor{blue!60} 6.7,\cellcolor{blue!35} 10.0,\cellcolor{blue!10} 10.0,NaN,NaN,NaN


In [16]:
def add_gpt_row(latex_str: str) -> str:
    lines: List[str] = []
    for line in latex_str.split('\n'):
        if re.match(f'^ *{COL_METHOD}.*', line):
            gpt_line = re.sub('\&[^\&]*', '& - ', line).replace(COL_METHOD, 'GPT-4') + '  \\\\'
            lines.append(line)
            lines.append(gpt_line)
        else:
            lines.append(line)
    return '\n'.join(lines)
    
latex_str = to_latex(colored_concat_df, with_index=True)
print(add_gpt_row(latex_str))

\begin{tabular}{lllllllllllll}
\toprule
 & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr \\
method &  &  &  &  &  &  &  &  &  &  &  &  \\
GPT-4 & - & - & - & - & - & - & - & - & - & - & - & -   \\
\midrule
matsuo-lab/weblab-10b & \cellcolor{blue!10} 18.3 & \cellcolor{blue!35} 43.3 & \cellcolor{blue!60} 76.7 & \cellcolor{blue!10} 0.0 & \cellcolor{blue!60} 5.0 & \cellcolor{blue!60} 23.3 & \cellcolor{blue!10} 0.0 & \cellcolor{blue!35} 10.0 & \cellcolor{blue!60} 30.0 & \cellcolor{blue!35} 0.0 & \cellcolor{blue!35} 0.0 & \cellcolor{blue!35} 6.7 \\
stabilityai/japanese-stablelm-base-alpha-7b & \cellcolor{blue!60} 20.0 & \cellcolor{blue!35} 43.3 & \cellcolor{blue!10} 60.0 & \cellcolor{blue!60} 3.3 & \cellcolor{blue!10} 3.3 & \cellcolor{blue!10} 0.0 & \cellcolor{blue!60} 6.7 & \cellcolor{blue!35} 10.0 & \cellcolor{blue!10} 10.0 & NaN & NaN & NaN \\
\bo

In [15]:
num_metrics = len(METRIC_RENAMES)
top_line = '{}    &    ' + '  &  '.join([f'\multicolumn{{{num_metrics}}}{{c}}{{{task_name}}}' for task_name in pretty_dfs.keys()]) + '    \\\\'
midrule_line = '    '.join(['\cmidrule(l{\\tabcolsep}r{\\tabcolsep})' + f'{{{num_metrics * (i_col + 1)}-{num_metrics * (i_col + 2) - 1}}}' for i_col in range(1, len(pretty_dfs) + 1)])

print(top_line.replace('_', '\_'))
print(midrule_line.replace('_', '\_'))

{}    &    \multicolumn{1}{c}{D1--.10}  &  \multicolumn{1}{c}{D1--.100}  &  \multicolumn{1}{c}{D1--.1000}  &  \multicolumn{1}{c}{D1-.10}  &  \multicolumn{1}{c}{D1-.100}  &  \multicolumn{1}{c}{D1-.1000}  &  \multicolumn{1}{c}{D1.10}  &  \multicolumn{1}{c}{D1.100}  &  \multicolumn{1}{c}{D1.1000}  &  \multicolumn{1}{c}{D3.10}  &  \multicolumn{1}{c}{D3.100}  &  \multicolumn{1}{c}{D3.1000}    \\
\cmidrule(l{\tabcolsep}r{\tabcolsep}){2-2}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){3-3}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){4-4}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){5-5}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){6-6}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){7-7}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){8-8}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){9-9}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){10-10}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){11-11}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){12-12}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){13-13}
